# Universal Monotonic Spectral Basis Exploration

This notebook explores the **Universal Monotonic Spectral Basis**, a sophisticated function basis designed for monotonic function approximation.

## Mathematical Foundation

The basis function is defined as:

$$
f(x) = c_0 \cdot x + \int_1^x \exp\left(a\log(t) + b + ct + dt^2 + e\exp(t) + g\sigma'(h(t-t_0)) + \sum_{k=1}^K d_k \cos(2\pi k \log(t))\right) dt
$$

where:
- $\sigma'(z) = \sigma(z)(1-\sigma(z))$ is the logistic sigmoid derivative
- Parameters control polynomial $(c, d)$, exponential $(e)$, sigmoid $(g, h, t_0)$, and spectral $(d_k)$ components
- **Guaranteed monotonic** for $x > 0$ because the integrand is always positive

## Key Properties

- **Trivializes standard functions**: Linear, logarithmic, exponential, sigmoid as special cases
- **Coarse-to-fine control**: Fourier terms in log-space provide hierarchical detail
- **Applications**: Calibration, propensity modeling, constrained approximation

## Cell 1: Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn

hv.extension('bokeh')
pn.extension()

import sys
sys.path.append('..')
from utils.monotonic_basis import (
    UniversalMonotonicBasis,
    BasisParameters,
    ParameterSampler
)

print("✓ All imports successful!")

## Cell 2: Generate Random Functions

Generate 15 diverse parameter sets and compute their corresponding functions.

In [ ]:
# Generate 15 random parameter sets
sampler = ParameterSampler(seed=42)
param_sets = sampler.sample_params(n_samples=15, mode='mixed')

# Evaluation domain [0.01, 1.0] (avoid x=0 due to log(0))
x_vals = np.linspace(0.01, 1.0, 200)

# Compute all functions
functions = []
for i, params in enumerate(param_sets):
    basis = UniversalMonotonicBasis(params)
    y = basis.evaluate(x_vals)
    functions.append({
        'x': x_vals,
        'y': y,
        'id': i,
        'params': params
    })

print(f"✓ Generated {len(functions)} diverse monotonic functions")

## Cell 3: Overlay Plot - Main Visualization

Interactive plot showing all 15 randomly generated monotonic functions.

In [ ]:
# Create DataFrame with all curves
dfs = []
for func in functions:
    df = pd.DataFrame({
        'x': func['x'],
        'f(x)': func['y'],
        'Curve': f"Curve {func['id']}"
    })
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

# Interactive hvplot with bokeh backend
overlay = combined_df.hvplot.line(
    x='x',
    y='f(x)',
    by='Curve',
    width=900,
    height=600,
    xlabel='x',
    ylabel='f(x)',
    title='Universal Monotonic Spectral Basis: Random Functions (x ∈ [0, 1])',
    legend='right',
    tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset'],
    responsive=False,
    line_width=2,
    alpha=0.7
)

overlay

## Cell 4: Interactive Parameter Explorer

Adjust sliders to see how each parameter affects the function shape in real-time.

The red line shows the derivative $f'(x)$, which should always be positive (above zero) for monotonicity.

In [ ]:
# Widget-based real-time exploration
@pn.depends(
    c_0=pn.widgets.FloatSlider(name='c₀ (linear)', start=0.1, end=5.0, step=0.1, value=1.0),
    a=pn.widgets.FloatSlider(name='a (log-power)', start=-2.0, end=2.0, step=0.1, value=0.0),
    b=pn.widgets.FloatSlider(name='b (offset)', start=-5.0, end=5.0, step=0.5, value=0.0),
    c=pn.widgets.FloatSlider(name='c (linear in exp)', start=-3.0, end=3.0, step=0.1, value=0.0),
    d=pn.widgets.FloatSlider(name='d (quadratic)', start=-2.0, end=2.0, step=0.1, value=0.0),
    e=pn.widgets.FloatSlider(name='e (exponential)', start=-1.0, end=1.0, step=0.1, value=0.0),
    g=pn.widgets.FloatSlider(name='g (sigmoid amp)', start=-3.0, end=3.0, step=0.1, value=0.0),
    h=pn.widgets.FloatSlider(name='h (sigmoid steepness)', start=0.5, end=10.0, step=0.5, value=1.0),
    t_0=pn.widgets.FloatSlider(name='t₀ (sigmoid center)', start=0.1, end=0.9, step=0.05, value=0.5),
)
def interactive_plot(c_0, a, b, c, d, e, g, h, t_0):
    params = BasisParameters(c_0=c_0, a=a, b=b, c=c, d=d, e=e, g=g, h=h, t_0=t_0)
    basis = UniversalMonotonicBasis(params)

    x = np.linspace(0.01, 1.0, 200)
    y = basis.evaluate(x)
    deriv = basis.derivative(x)

    df = pd.DataFrame({'x': x, 'f(x)': y, "f'(x)": deriv})

    # Plot both function and derivative
    func_plot = df.hvplot.line(x='x', y='f(x)', label='f(x)', color='blue', line_width=2)
    deriv_plot = df.hvplot.line(x='x', y="f'(x)", label="f'(x)", color='red', line_width=2)

    is_monotonic = basis.verify_monotonic(x)

    return (func_plot * deriv_plot).opts(
        width=900,
        height=500,
        title=f"Monotonic: {is_monotonic} | f'(x) > 0 required",
        legend_position='top_right',
        tools=['hover', 'pan', 'wheel_zoom', 'reset']
    )

pn.Column(
    "## Interactive Parameter Explorer",
    "Adjust sliders to see how each parameter affects the function shape.",
    interactive_plot
)

## Cell 5: Function Value Distributions

Statistical analysis of the generated functions at key points.

In [ ]:
# Statistical analysis of generated functions
values_at_05 = [
    UniversalMonotonicBasis(p).evaluate(np.array([0.5]))[0]
    for p in param_sets
]
values_at_10 = [
    UniversalMonotonicBasis(p).evaluate(np.array([1.0]))[0]
    for p in param_sets
]

stats_df = pd.DataFrame({
    'f(0.5)': values_at_05,
    'f(1.0)': values_at_10
})

# Histogram with hvplot
hist = stats_df.hvplot.hist(
    bins=15,
    alpha=0.6,
    width=900,
    height=400,
    title='Distribution of Function Values at x=0.5 and x=1.0',
    ylabel='Count',
    legend='top_right'
)

hist

## Cell 6: Special Case Demonstrations

The basis trivializes common monotonic functions as special cases:
- **Linear**: $c_0 = 1$, all other parameters = 0
- **Log-like**: $a = 1$, $b = -1$, $c_0 = 0$
- **Sigmoid step**: Large $g$, large $h$ for sharp transition

In [ ]:
# Show that basis trivializes standard functions

# Linear: f(x) = x
linear_params = BasisParameters(c_0=1.0)

# Logarithmic dominance
log_params = BasisParameters(c_0=0.0, a=1.0, b=-1.0)

# Sigmoid step
sigmoid_params = BasisParameters(c_0=0.0, g=5.0, h=10.0, t_0=0.5, b=0.5)

special_cases = [
    ('Linear (c₀=1)', linear_params),
    ('Log-like (a=1)', log_params),
    ('Sigmoid step', sigmoid_params)
]

x = np.linspace(0.01, 1.0, 200)
special_dfs = []

for name, params in special_cases:
    basis = UniversalMonotonicBasis(params)
    y = basis.evaluate(x)
    df = pd.DataFrame({'x': x, 'f(x)': y, 'Type': name})
    special_dfs.append(df)

special_df = pd.concat(special_dfs, ignore_index=True)

special_plot = special_df.hvplot.line(
    x='x',
    y='f(x)',
    by='Type',
    width=900,
    height=500,
    title='Special Cases: Trivializing Standard Functions',
    legend='top_left',
    line_width=3
)

special_plot

## Cell 7: Monotonicity Verification

Verify that all generated functions are truly monotonic by checking that $f'(x) \geq 0$ everywhere.

In [ ]:
# Verify all generated functions are truly monotonic
x_test = np.linspace(0.01, 1.0, 100)

results = []
for i, params in enumerate(param_sets):
    basis = UniversalMonotonicBasis(params)
    is_monotonic = basis.verify_monotonic(x_test)
    min_deriv = basis.derivative(x_test).min()
    max_deriv = basis.derivative(x_test).max()

    results.append({
        'Curve ID': i,
        'Monotonic': '✓' if is_monotonic else '✗',
        "Min f'(x)": f'{min_deriv:.6f}',
        "Max f'(x)": f'{max_deriv:.6f}'
    })

verification_df = pd.DataFrame(results)
print("\nMonotonicity Verification Results:")
print("All curves should have Min f'(x) ≥ 0\n")

# Display as table
verification_df.hvplot.table(
    width=700,
    height=400,
    title="Monotonicity Verification (all should show ✓)"
)